In [ ]:
# hide
%load_ext autoreload
%autoreload 2

# Build end-to-end Vespa apps with pyvespa

> Python API to create, modify, deploy and interact with Vespa applications

Our goal is to create, modify and deploy simple application packages using our python API. This enables us to run data analysis experiments that are fully integrated with Vespa. As an example, we want to create the application package we used in our [text search tutorial](https://docs.vespa.ai/documentation/tutorials/text-search.html). 

## Schema API

In [ ]:
from vespa.package import Document, Field, Schema, FieldSet, RankProfile, ApplicationPackage

document = Document(
    fields=[
        Field(name = "id", type = "string", indexing = ["attribute", "summary"]),
        Field(name = "title", type = "string", indexing = ["index", "summary"], index = "enable-bm25"),
        Field(name = "body", type = "string", indexing = ["index", "summary"], index = "enable-bm25")        
    ]
)

msmarco_schema = Schema(
    name = "msmarco", 
    document = document, 
    fieldsets = [FieldSet(name = "default", fields = ["title", "body"])],
    rank_profiles = [RankProfile(name = "default", first_phase = "nativeRank(title, body)")]
)

app_package = ApplicationPackage(name = "msmarco", schema=msmarco_schema)

## Deploy it locally

In [ ]:
from vespa.package import VespaDocker

vespa_docker = VespaDocker(application_package=app_package)
deployment_msg, app = vespa_docker.deploy(disk_folder="/Users/tmartins/projects/vespa/vespa/python/vespa/notebooks/sample_application")

In [ ]:
deployment_msg

["Uploading application '/app/application' using http://localhost:19071/application/v2/tenant/default/session",
 "Session 7 for tenant 'default' created.",
 'Preparing session 7 using http://localhost:19071/application/v2/tenant/default/session/7/prepared',
 "WARNING: Host named 'msmarco' may not receive any config since it is not a canonical hostname. Disregard this warning when testing in a Docker container.",
 "Session 7 for tenant 'default' prepared.",
 'Activating session 7 using http://localhost:19071/application/v2/tenant/default/session/7/active',
 "Session 7 for tenant 'default' activated.",
 'Checksum:   09203c16fa5f582b712711bb98932812',
 'Timestamp:  1597839235945',
 'Generation: 7',
 '']

In [ ]:
print(app)

Vespa(http://localhost:8080)


## Change the application package and redeploy

We can add a new rank profile and redeploy our application

In [ ]:
app_package.schema.add_rank_profile(
    RankProfile(name = "bm25", inherits = "default", first_phase = "bm25(title) + bm25(body)")
)